In [70]:
import requests
import pandas as pd
import pytz
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [71]:
def get_tide_data(station_id, start_date, end_date, product="predictions", datum="MLLW", time_zone="lst", interval="hilo"):
    """
    Pulls tidal data from the NOAA CO-OPS API.

    Args:
        station_id (str): The 7-character NOAA tide station ID.
        start_date (str): Start date in YYYYMMDD format.
        end_date (str): End date in YYYYMMDD format.
        product (str): Type of data (e.g., "predictions", "high_low").
        datum (str): Tidal datum (e.g., "MLLW", "MSL").
        time_zone (str): Time zone for data (e.g., "lst", "gmt").
        interval (str): Interval for predictions (e.g., "hilo" for high/low, "h" for hourly).

    Returns:
        pandas.DataFrame: A DataFrame containing the tidal data, or None if an error occurs.
    """
    base_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?"

    params = {
        "product": product,
        "application": "PythonSharptownTideTracker",
        "station": station_id,
        "begin_date": start_date,
        "end_date": end_date,
        "datum": datum,
        "units": "english",  # or "metric"
        "time_zone": time_zone,
        "interval": interval,
        "format": "json"
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        if "predictions" in data:
            df = pd.DataFrame(data["predictions"])
            df['t'] = pd.to_datetime(df['t'], utc=False).dt.tz_localize(None)
            df.rename(columns={'t': 'datetime', 'v': 'height_ft'}, inplace=True)

            df['height_ft'] = pd.to_numeric(df['height_ft'], errors='coerce')
            df.dropna(subset=['height_ft'], inplace=True)

            if 'type' in df.columns:
                df.rename(columns={'type': 'tide_type'}, inplace=True)
                return df[['datetime', 'tide_type', 'height_ft']]
            else:
                df['tide_type'] = ''
                return df[['datetime', 'tide_type', 'height_ft']]
        elif "data" in data:
            df = pd.DataFrame(data["data"])
            df['t'] = pd.to_datetime(df['t'])
            # --- NEW ROBUST FIX: Strip timezone information if it exists ---
            if df['t'].dt.tz is not None:
                df['t'] = df['t'].dt.tz_localize(None)
            # --- END NEW ROBUST FIX ---
            df.rename(columns={'t': 'datetime', 'v': 'height_ft'}, inplace=True)
            df['height_ft'] = pd.to_numeric(df['height_ft'], errors='coerce')
            df.dropna(subset=['height_ft'], inplace=True)
            df['tide_type'] = ''
            return df[['datetime', 'height_ft', 'tide_type']]
        else:
            print(f"No tidal data found for station {station_id} with the given parameters.")
            print(f"API Response: {data}")
            return None

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response content: {response.text}")
        return None
    except requests.exceptions.ConnectionError as e:
        print(f"Connection Error: {e}")
        return None
    except requests.exceptions.Timeout as e:
        print(f"Timeout Error: {e}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Response content: {response.text}")
        return None

In [72]:
def get_pirate_weather_report(api_key, latitude, longitude, time_unix, units="us"):
    """
    Pulls weather data from the Pirate Weather API for a specific time and location.

    Args:
        api_key (str): Your Pirate Weather API key.
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        time_unix (int): Unix timestamp for the desired weather report.
        units (str): Units for the weather data (e.g., "us", "si", "ca", "uk").

    Returns:
        dict: A dictionary containing the weather data, or None if an error occurs.
    """
    # Pirate Weather API endpoint for forecast at a specific time
    # The [time] parameter takes a Unix timestamp
    url = f"https://api.pirateweather.net/forecast/{api_key}/{latitude},{longitude},{time_unix}"

    params = {
        "units": units,
        "exclude": "minutely,daily,alerts,flags" # Exclude unnecessary data for a focused report
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.HTTPError as e:
        print(f"Pirate Weather HTTP Error: {e}")
        print(f"Pirate Weather Response content: {response.text}")
        return None
    except requests.exceptions.ConnectionError as e:
        print(f"Pirate Weather Connection Error: {e}")
        return None
    except requests.exceptions.Timeout as e:
        print(f"Pirate Weather Timeout Error: {e}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred with Pirate Weather: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing Pirate Weather JSON: {e}")
        print(f"Pirate Weather Response content: {response.text}")
        return None

In [73]:
def plot_tide_data(df, station_name="Tide Station", start_date=None, end_date=None):
    """
    Generates a plot of tide heights over time, marking high and low tides.

    Args:
        df (pandas.DataFrame): DataFrame containing 'datetime', 'height_ft', and 'tide_type' columns.
        station_name (str): Name of the tide station for the plot title.
        start_date (datetime): The start date of the data for plot labeling.
        end_date (datetime): The end date of the data for plot labeling.
    """
    if df is None or df.empty:
        print("No data to plot.")
        return

    plt.figure(figsize=(14, 7))

    # Plot the hourly data (or the main continuous line)
    # This assumes 'tide_type' is empty for the continuous hourly data
    hourly_plot_df = df[df['tide_type'] == '']
    if not hourly_plot_df.empty:
        plt.plot(hourly_plot_df['datetime'], hourly_plot_df['height_ft'], linestyle='-', marker='', color='blue', label='Tide Height')
    else:
        # Fallback if only high/low points are available, plot them as a dashed line
        plt.plot(df['datetime'], df['height_ft'], linestyle=':', marker='o', color='gray', alpha=0.5, label='High/Low Points')


    # Mark High and Low Tides
    high_tides = df[df['tide_type'] == 'H']
    low_tides = df[df['tide_type'] == 'L']

    plt.scatter(high_tides['datetime'], high_tides['height_ft'], color='red', s=50, zorder=5, label='High Tide')
    plt.scatter(low_tides['datetime'], low_tides['height_ft'], color='green', s=50, zorder=5, label='Low Tide')

    # Annotate high and low tides
    for index, row in high_tides.iterrows():
        if pd.notna(row['height_ft']):
            plt.text(row['datetime'], row['height_ft'] + 0.1, f"H: {row['height_ft']:.1f} ft\n{row['datetime'].strftime('%I:%M %p')}",
                     fontsize=8, ha='center', va='bottom', color='red')
    for index, row in low_tides.iterrows():
        if pd.notna(row['height_ft']):
            plt.text(row['datetime'], row['height_ft'] - 0.1, f"L: {row['height_ft']:.1f} ft\n{row['datetime'].strftime('%I:%M %p')}",
                     fontsize=8, ha='center', va='top', color='green')


    plt.title(f"Tide Predictions for {station_name}")
    plt.xlabel("Date and Time")
    plt.ylabel("Tide Height (feet)")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # Format x-axis for better date/time display
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %I:%M %p'))
    plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=3)) # Show every 3 hours
    plt.xticks(rotation=45, ha='right')

    plt.tight_layout()
    plt.show()

In [74]:
# --- Configuration for Sharptown, MD ---
your_station_id = "8571858" # Station ID for Sharptown, Nanticoke River, MD
station_name = "Sharptown, Nanticoke River, MD"
LOCAL_TIMEZONE = pytz.timezone('America/New_York')

# Sharptown, MD Coordinates (Approximate, you can refine this with a map)
# Using the coordinates from the NOAA station page for Sharptown:
# Lat: 38.3970, Lon: -75.7600
sharptown_latitude = 38.3970
sharptown_longitude = -75.7600


# !!! IMPORTANT: Replace with your actual Pirate Weather API Key !!!
PIRATE_WEATHER_API_KEY = "lmTROGhlgrRkf7IzzTOm37PcyPg4xKoK"
if PIRATE_WEATHER_API_KEY == "YOUR_PIRATE_WEATHER_API_KEY":
    print("WARNING: Please replace 'YOUR_PIRATE_WEATHER_API_KEY' with your actual API key from pirate-weather.apiable.io to get weather data.")


# Define the date range (e.g., today and the next two days for a good plot)
today = datetime.now() # Get current actual time
end_date = today + timedelta(days=2) # Get data for today and the next two days
start_date_str = today.strftime("%Y%m%d")
end_date_str = end_date.strftime("%Y%m%d")


In [75]:
# --- Get High/Low Tide Predictions (for labeling points on graph and weather lookup) ---
print(f"Fetching high/low tide predictions for {station_name} from {start_date_str} to {end_date_str}...")
hilo_tide_predictions_df = get_tide_data(
    station_id=your_station_id,
    start_date=start_date_str,
    end_date=end_date_str,
    product="predictions",
    datum="MLLW",
    time_zone="lst",
    interval="hilo" # Request high/low predictions
)
# APPLY tz_localize HERE for hilo_tide_predictions_df
if hilo_tide_predictions_df is not None:
    hilo_tide_predictions_df['datetime'] = hilo_tide_predictions_df['datetime'].dt.tz_localize(LOCAL_TIMEZONE, ambiguous='infer', nonexistent='shift_forward')

# --- Get Hourly Tide Predictions (for plotting the continuous curve) ---
print(f"\nFetching hourly tide predictions for {station_name} from {start_date_str} to {end_date_str} (for plotting)...")
hourly_predictions_df = get_tide_data(
    station_id=your_station_id,
    start_date=start_date_str,
    end_date=end_date_str,
    product="predictions",
    datum="MLLW",
    time_zone="lst",
    interval="h" # Request hourly predictions for the continuous line
)
if hilo_tide_predictions_df is not None:
    hilo_tide_predictions_df['datetime'] = hilo_tide_predictions_df['datetime'].dt.tz_localize(LOCAL_TIMEZONE, ambiguous='infer', nonexistent='shift_forward')

# --- Process and Display Tide Predictions (Text Output) ---
if hilo_tide_predictions_df is not None:
    print("\n--- High and Low Tide Predictions for Sharptown, MD ---")
    print(hilo_tide_predictions_df.to_string(index=False))

    current_time = datetime.now() # Use current time for "next tide" calculation
    # Filter for tides in the future from the hilo_tide_predictions_df
    future_tides = hilo_tide_predictions_df[hilo_tide_predictions_df['datetime'] > current_time].copy() # Use .copy() to avoid SettingWithCopyWarning

    # --- Get Weather Reports for High Tides ---
    print("\n--- Pirate Weather Reports for High Tides ---")
    if PIRATE_WEATHER_API_KEY != "YOUR_PIRATE_WEATHER_API_KEY" and not future_tides.empty:
        high_tides_for_weather = future_tides[future_tides['tide_type'] == 'H']
        if not high_tides_for_weather.empty:
            for index, row in high_tides_for_weather.iterrows():
                tide_time_utc = row['datetime'].astimezone(timezone.utc) # Convert to UTC for Unix timestamp
                unix_timestamp = int(tide_time_utc.timestamp())

                print(f"\nWeather at High Tide ({row['datetime'].strftime('%Y-%m-%d %I:%M %p %Z')}):")
                weather_data = get_pirate_weather_report(
                    PIRATE_WEATHER_API_KEY,
                    sharptown_latitude,
                    sharptown_longitude,
                    unix_timestamp
                )
                if weather_data and 'currently' in weather_data:
                    current_weather = weather_data['currently']
                    print(f"  Summary: {current_weather.get('summary', 'N/A')}")
                    print(f"  Temperature: {current_weather.get('temperature', 'N/A')}°F")
                    print(f"  Feels Like: {current_weather.get('apparentTemperature', 'N/A')}°F")
                    print(f"  Precipitation Probability: {current_weather.get('precipProbability', 'N/A') * 100:.0f}%")
                    print(f"  Wind Speed: {current_weather.get('windSpeed', 'N/A')} mph")
                    print(f"  Humidity: {current_weather.get('humidity', 'N/A') * 100:.0f}%")
                    # You can add more weather details as needed from the 'currently' object
                    # print(f"Full weather data: {json.dumps(weather_data, indent=2)}") # Uncomment to see full response
                else:
                    print("  Could not retrieve detailed weather data for this time.")
        else:
            print("No future High Tides found to fetch weather for.")
    elif PIRATE_WEATHER_API_KEY == "YOUR_PIRATE_WEATHER_API_KEY":
        print("Skipping Pirate Weather requests. Please provide your API key.")
    else:
        print("No future tides to fetch weather for.")


    # --- Find and Display Next High/Low Tide (Text Output, after weather) ---
    if not future_tides.empty:
        if not future_tides[future_tides['tide_type'] == 'H'].empty:
            next_high_tide = future_tides[future_tides['tide_type'] == 'H'].sort_values(by='datetime').iloc[0]
            if pd.notna(next_high_tide['height_ft']):
                print(f"\nNext High Tide: {next_high_tide['datetime'].strftime('%Y-%m-%d %I:%M %p %Z')} (Height: {next_high_tide['height_ft']:.2f} ft)")
            else:
                print(f"\nNext High Tide: {next_high_tide['datetime'].strftime('%Y-%m-%d %I:%M %p %Z')} (Height: N/A ft)")
        else:
            print("\nNo future High Tides found in the data.")

        if not future_tides[future_tides['tide_type'] == 'L'].empty:
            next_low_tide = future_tides[future_tides['tide_type'] == 'L'].sort_values(by='datetime').iloc[0]
            if pd.notna(next_low_tide['height_ft']):
                print(f"Next Low Tide: {next_low_tide['datetime'].strftime('%Y-%m-%d %I:%M %p %Z')} (Height: {next_low_tide['height_ft']:.2f} ft)")
            else:
                print(f"Next Low Tide: {next_low_tide['datetime'].strftime('%Y-%m-%d %I:%M %p %Z')} (Height: N/A ft)")
        else:
            print("No future Low Tides found in the data.")
    else:
        print("\nNo future tide predictions available for the specified date range.")
else:
    print("Failed to retrieve high/low tide data for text output.")


# --- Plot the Data ---
if hourly_predictions_df is not None and hilo_tide_predictions_df is not None:
    # Concatenate and sort, then drop duplicates (hourly and hilo points might overlap exactly)
    combined_df = pd.concat([hourly_predictions_df, hilo_tide_predictions_df], ignore_index=True)
    combined_df = combined_df.sort_values(by='datetime').drop_duplicates(subset=['datetime', 'height_ft'])

    plot_tide_data(
        combined_df,
        station_name=station_name,
        start_date=today,
        end_date=end_date
    )
elif hourly_predictions_df is not None:
    print("\nNote: Only hourly data available for plotting. High/Low marks may not be explicit.")
    plot_tide_data(
        hourly_predictions_df,
        station_name=station_name,
        start_date=today,
        end_date=end_date
    )
else:
    print("Could not retrieve enough data for plotting.")

Fetching high/low tide predictions for Sharptown, Nanticoke River, MD from 20250531 to 20250602...

Fetching hourly tide predictions for Sharptown, Nanticoke River, MD from 20250531 to 20250602 (for plotting)...


TypeError: Already tz-aware, use tz_convert to convert.